In [ ]:
import tensorflow as tf
print(tf.__version__)
# More imports
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from tensorflow.keras.layers import Input, SimpleRNN, GRU, LSTM, Dense, Flatten, Bidirectional, Conv1D, concatenate, Permute, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2.8.0


In [ ]:
mnist = tf.keras.datasets.fashion_mnist

(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

#dividing each cell by 255 as the pixel value ranges from 0-255
x_train_full, x_test = x_train_full / 255.0, x_test / 255.0
print("x_train_full.shape:", x_train_full.shape)
#print("x_train_full.shape:", x_train_full.shape)
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)
x_train, y_train = x_train_full, y_train_full
x_train, x_valid = x_train_full[:50000], x_train_full[50000:]
y_train, y_valid = y_train_full[:50000], y_train_full[50000:]
print("x_train.shape:", x_train.shape)
print("y_train.shape:", y_train.shape)
print("x_valid.shape:", x_valid.shape)
print("y_valid.shape:", y_valid.shape)

x_train_full.shape: (60000, 28, 28)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)
x_train.shape: (50000, 28, 28)
y_train.shape: (50000,)
x_valid.shape: (10000, 28, 28)
y_valid.shape: (10000,)


Bidirectional LSTM

In [ ]:
# batch_size = 32
num_classes = 10
# epochs = 10

# row_hidden = 128
# col_hidden = 128

row, col = x_train_full.shape[1:]
print(row,col)

input = Input(shape=(row, col))

def lstm_pipe(in_layer,row_hidden):
    encoded_rows = Bidirectional(LSTM(row_hidden, return_sequences = True))(in_layer)
    return encoded_rows

def model_lstm(row_hidden):
    # read it by rows to read a picture from left to right
    first_read = lstm_pipe(input,row_hidden)

    # read it by columns to read a picture from top to bottom, we need to transpose it and then read
    #after transposing passing the picture through the Bi-LSTM layer
    trans_read = lstm_pipe(Permute(dims = (1,2))(input),row_hidden)

    #concatenate the out of bi-lstm for both normal as well as transpose matrix
    encoded_columns = concatenate([first_read, trans_read])
    encoded_columns = Dropout(0.2)(encoded_columns)
    
    print(tf.shape(encoded_columns))
    
    #to pass to a dense layer we need to flatten the vector
    encoded_columns = Flatten()(encoded_columns)
    print(tf.shape(encoded_columns))

    #softmax is being used as we need only 1 class out of 10 classes
    prediction = Dense(num_classes, activation='softmax')(encoded_columns)
    model = Model(input, prediction)
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    model.summary()
    return model

28 28


In [ ]:
#executing the gridsearch with different batch size, epochs and hidden rows
grid_param_LSTM = {
    'batch_size': [32,64],
    'epochs': [5,10,30], 
    'row_hidden':[128,256]
}

model=KerasClassifier(build_fn=model_lstm)

GridLSTM = GridSearchCV(estimator=model,
                     param_grid=grid_param_LSTM,
                     cv=2)


GridLSTM.fit(x_train, y_train)

print('Best Params of Grid Search')
GridLSTM.best_params_

print('Grid Search Results')
keys = []
shapes = []
measures = []
for key in list(GridLSTM.cv_results_.keys()):
    keys.append(key)
    shapes.append(len(GridLSTM.cv_results_.get(key)))
    measures.append(GridLSTM.cv_results_.get(key)[0])

df_results_info = pd.DataFrame({'position': shapes, 'values': measures}, index = keys)
df_results_info

y_pred = GridLSTM.predict(x_test)


print('Accuracy Score')
print(accuracy_score(y_test, y_pred))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


KerasTensor(type_spec=TensorSpec(shape=(3,), dtype=tf.int32, name=None), inferred_value=[None, 28, 512], name='tf.compat.v1.shape/Shape:0', description="created by layer 'tf.compat.v1.shape'")
KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 14336], name='tf.compat.v1.shape_1/Shape:0', description="created by layer 'tf.compat.v1.shape_1'")
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 permute (Permute)              (None, 28, 28)       0           ['input_1[0][0]']                
                                                                                                  
 bi

In [ ]:
GridLSTM.best_params_

{'batch_size': 32, 'epochs': 10}

In [ ]:
GridLSTM.cv_results_

{'mean_fit_time': array([149.96511388, 308.44172323,  85.44267249, 210.06663764]),
 'mean_score_time': array([17.88725424, 13.44064188,  7.6261524 ,  9.76178145]),
 'mean_test_score': array([0.87564   , 0.88854   , 0.86979997, 0.8849    ]),
 'param_batch_size': masked_array(data=[32, 32, 64, 64],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[5, 10, 5, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'batch_size': 32, 'epochs': 5},
  {'batch_size': 32, 'epochs': 10},
  {'batch_size': 64, 'epochs': 5},
  {'batch_size': 64, 'epochs': 10}],
 'rank_test_score': array([3, 1, 4, 2], dtype=int32),
 'split0_test_score': array([0.87548   , 0.89024001, 0.86799997, 0.88704002]),
 'split1_test_score': array([0.87580001, 0.88683999, 0.87159997, 0.88275999]),
 'std_fit_time': array([3.59181404e+00, 2.11488646e+01, 4.61761642e+00, 1.06941462e-

In [ ]:
y_pred = GridLSTM.predict(x_test)

In [ ]:
print('Accuracy Score')
print(accuracy_score(y_test, y_pred))

Accuracy Score
0.8941


Bidirectional GRU

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, Bidirectional, Conv1D, concatenate, Permute, Dropout

batch_size = 32
num_classes = 10
epochs = 7

row_hidden = 128
col_hidden = 128

row, col = x_train_full.shape[1:]
print(row,col)

input = Input(shape=(row, col))

def gru_pipe(in_layer):
    encoded_rows = Bidirectional(GRU(row_hidden, return_sequences = True))(in_layer)
    return encoded_rows

# read it by rows
first_read = gru_pipe(input)
# read it by columns to read a picture from top to bottom, we need to transpose it and then read
# after transposing passing the picture through the Bi-LSTM layer
trans_read = gru_pipe(Permute(dims = (1,2))(input))

#concatenate the out of bi-lstm for both normal as well as transpose matrix
encoded_columns = concatenate([first_read, trans_read])
encoded_columns = Dropout(0.2)(encoded_columns)

print(tf.shape(encoded_columns))

#to pass to a dense layer we need to flatten the vector
encoded_columns = Flatten()(encoded_columns)
print(tf.shape(encoded_columns))

#softmax is being used as we need only 1 class out of 10 classes
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model_gru = Model(input, prediction)
model_gru.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_gru.summary()

28 28
KerasTensor(type_spec=TensorSpec(shape=(3,), dtype=tf.int32, name=None), inferred_value=[None, 28, 512], name='tf.compat.v1.shape_4/Shape:0', description="created by layer 'tf.compat.v1.shape_4'")
KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 14336], name='tf.compat.v1.shape_5/Shape:0', description="created by layer 'tf.compat.v1.shape_5'")
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 permute_2 (Permute)            (None, 28, 28)       0           ['input_3[0][0]']                
                                                                                          

In [ ]:
history = model_gru.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_valid, y_valid))

Epoch 1/7
1563/1563 [==============================] - 65s 37ms/step - loss: 0.4960 - accuracy: 0.8256 - val_loss: 0.4269 - val_accuracy: 0.8465
Epoch 2/7
1563/1563 [==============================] - 56s 36ms/step - loss: 0.3848 - accuracy: 0.8626 - val_loss: 0.3721 - val_accuracy: 0.8668
Epoch 3/7
1563/1563 [==============================] - 57s 36ms/step - loss: 0.3364 - accuracy: 0.8783 - val_loss: 0.3421 - val_accuracy: 0.8784
Epoch 4/7
1563/1563 [==============================] - 57s 36ms/step - loss: 0.3030 - accuracy: 0.8906 - val_loss: 0.3396 - val_accuracy: 0.8789
Epoch 5/7
1563/1563 [==============================] - 57s 36ms/step - loss: 0.2814 - accuracy: 0.8979 - val_loss: 0.3332 - val_accuracy: 0.8877
Epoch 6/7
1563/1563 [==============================] - 57s 37ms/step - loss: 0.2616 - accuracy: 0.9059 - val_loss: 0.3158 - val_accuracy: 0.8921
Epoch 7/7
1563/1563 [==============================] - 56s 36ms/step - loss: 0.2405 - accuracy: 0.9116 - val_loss: 0.3100 - val_ac

In [ ]:
y_pred_gru = model_gru.predict(x_test)

In [ ]:
scores_gru = model_gru.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores_gru[0])
print('Test accuracy:', scores_gru[1])

Test loss: 0.3221788704395294
Test accuracy: 0.8932999968528748


Multiple Layers of LSTM

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM, Bidirectional, Conv1D, concatenate, Permute, Dropout

batch_size = 32
num_classes = 10
epochs = 7

row_hidden = 128
col_hidden = 128

row, col = x_train_full.shape[1:]
print(row,col)

input = Input(shape=(row, col))

def multi_lstm_pipe(in_layer):
    encoded_rows = Bidirectional(LSTM(row_hidden, return_sequences = True))(in_layer)
    encoded_rows = Bidirectional(LSTM(row_hidden, return_sequences = True))(encoded_rows)
    return encoded_rows

# read it by rows
first_read = multi_lstm_pipe(input)

# read it by columns to read a picture from top to bottom, we need to transpose it and then read
#after transposing passing the picture through the Bi-LSTM layer
trans_read = multi_lstm_pipe(Permute(dims = (1,2))(input))

#concatenate the out of bi-lstm for both normal as well as transpose matrix
encoded_columns = concatenate([first_read, trans_read])
encoded_columns = Dropout(0.2)(encoded_columns)

print(tf.shape(encoded_columns))

#to pass to a dense layer we need to flatten the vector
encoded_columns = Flatten()(encoded_columns)
print(tf.shape(encoded_columns))

#softmax is being used as we need only 1 class out of 10 classes
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model_multi_lstm = Model(input, prediction)
model_multi_lstm.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_multi_lstm.summary()

28 28
KerasTensor(type_spec=TensorSpec(shape=(3,), dtype=tf.int32, name=None), inferred_value=[None, 28, 512], name='tf.compat.v1.shape_6/Shape:0', description="created by layer 'tf.compat.v1.shape_6'")
KerasTensor(type_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None), inferred_value=[None, 14336], name='tf.compat.v1.shape_7/Shape:0', description="created by layer 'tf.compat.v1.shape_7'")
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 permute_3 (Permute)            (None, 28, 28)       0           ['input_4[0][0]']                
                                                                                          

In [ ]:
history = model_multi_lstm.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_valid, y_valid))

Epoch 1/7
1563/1563 [==============================] - 138s 80ms/step - loss: 0.5068 - accuracy: 0.8155 - val_loss: 0.4178 - val_accuracy: 0.8489
Epoch 2/7
1563/1563 [==============================] - 121s 77ms/step - loss: 0.3715 - accuracy: 0.8655 - val_loss: 0.3537 - val_accuracy: 0.8697
Epoch 3/7
1563/1563 [==============================] - 122s 78ms/step - loss: 0.3206 - accuracy: 0.8829 - val_loss: 0.3017 - val_accuracy: 0.8945
Epoch 4/7
1563/1563 [==============================] - 119s 76ms/step - loss: 0.2882 - accuracy: 0.8923 - val_loss: 0.3053 - val_accuracy: 0.8886
Epoch 5/7
1563/1563 [==============================] - 120s 77ms/step - loss: 0.2648 - accuracy: 0.9006 - val_loss: 0.2838 - val_accuracy: 0.8956
Epoch 6/7
1563/1563 [==============================] - 122s 78ms/step - loss: 0.2420 - accuracy: 0.9098 - val_loss: 0.2866 - val_accuracy: 0.8985
Epoch 7/7
1563/1563 [==============================] - 122s 78ms/step - loss: 0.2230 - accuracy: 0.9163 - val_loss: 0.2621 -

In [ ]:
y_pre_multi_lstm = model_multi_lstm.predict(x_test)

In [ ]:
scores_multi_lstm = model_multi_lstm.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores_gru[0])
print('Test accuracy:', scores_gru[1])

Test loss: 0.3221788704395294
Test accuracy: 0.8932999968528748
